In [1]:
import numpy as np 
import pandas as pd 
import datetime as datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import clear_output,display
from datetime import *
from ib_async import *
import asyncio
import time
from time import sleep


In [ ]:
from ib_async import *
util.startLoop()
ib = IB()
ib.connect(port =  7497, clientId = 0)

ib.updateEvent.clear()

In [ ]:
contract = Stock(symbol = 'NVDA', exchange = 'SMART', currency = 'USD')
ib.qualifyContracts(contract)
contract

In [ ]:
details = ib.reqContractDetails(contract)
print(details)
details[0].tradingHours

In [ ]:
bars = ib.reqHistoricalData(contract = contract, endDateTime='', durationStr= '1 D', barSizeSetting='30 secs', whatToShow='TRADES', useRTH=False, keepUpToDate=True)

In [ ]:
bars

In [ ]:
util.df(bars)

In [ ]:
#util.df(bars).assign(ADD_NEW_COLUMN = 'Whatever you wanT')

In [ ]:
from IPython.display import clear_output,display
from datetime import *
def onBarUpdate(bars, hasNewBars):
    # clear_output()
    display(util.df(bars).tail(2).assign(New_bar = hasNewBars, Update_Time = datetime.now()))

# Every time there is a change in the bars the function is ran

bars.updateEvent.clear()
# or bars.updateEvent -= onBarUpdate unsubscribes the callback
bars.updateEvent += onBarUpdate
ib.sleep(60)
bars.updateEvent.clear()


,date,open,high,low,close,volume,average,barCount,New_bar,Update_Time
1175,2024-11-01 13:47:30-04:00,135.82,135.82,135.76,135.76,57871.0,135.797307,143,False,2024-11-01 17:48:24.455009
1176,2024-11-01 13:48:00-04:00,135.76,135.77,135.70,135.75,56483.0,135.729717,206,False,2024-11-01 17:48:24.455009


Say we want the output to only have full bars

In [ ]:
def onBarUpdateFull(bars, hasNewBars):
    if hasNewBars:
        display(util.df(bars).iloc[-2:].assign(Update_time = datetime.now()))

bars.updateEvent.clear()
bars.updateEvent += onBarUpdateFull
ib.sleep(120)
bars.updateEvent.clear()

# Bollinger Bands

Text taken and typed out from 'The New Trading for a Living' (Elder, 2014) with intention of learning and improving touch typing.

## Channel Trading systems

Market prices tend to flow in channels, like a river in their valleys. When a river touches the right edge of its valley, it turns left. When it touches the left rim of its valley it turns right.

When prices rally, they often seem to stop at an invisible ceiling. Their declines seem to stop at invisible floors. Channels help us anticipate where their support and resistance levels are likely to be encountered. Support is where buyers buy with greater intensity than sellers sell. Resistance is where sellers sell with greater intesnity than buyers buy. Channels show where to expect support and resistance in the future. Channels help identify buying and selling opportunities and avoid bad trades. The late great mathematician Benoit Mandelbrot was hired by the Egyptian government to create a mathematical model of cotton prices - the main agricultural export of that country. After extensive study, the scientist made the finding, 'prices oscillate above and below a value' It may sound simple, but it is in fact profound. If we accept this mathematical finding and if we have the means to define value and measure an average oscillation, we'll have a trading system. We'll need to buy below value and take profits at value or sell short above value and cover at value. We have already agreed that value is in the zone between a short and a long moving average. We can use channels to find normal and abnormal oscillations. 

We construct a channel by plotting two lines parralel to a moving average, one above and another below. We may also vary the distance between the channel lines depending on that market's volatility (standard deviation channels). A symmetrical channel, centered around a moving average, is useful for trading stocks. A standard deviation channel (sometimes called Bollinger bands) is used also.

Channels mark the boundaries between normal and abnormal price action. It is normal for prices to stay inside a well-drawn channel, and only unusual events push them outside. The market is undervalued below its lower channel line and overvalued above its upper channel line. 

__Market Psychology__

An exponential moving average reflects the average consensus of value in its time window. When prices are near their moving average, the market is fairly valued. When they decline near the lower channel line, the market is undervalued. When prices rise to the upper channel line, the market is overvalued. Channels help find buying opportunities when the market is cheap and shorting opportunities when the market is dear. When prices fall below their moving average, bargain hunters step in. Their buying, as well as short covering by bears stops declines and lifts prices. When prices rise above value, sellers see an opportunity to take profits on long positions or go short, their selling caps the rise. When the market sinks to the bottom of a depression, its mood is about to improve. Once it rises to the height of its mania, it's about to start calming down. A channel marks normal limits of mass optimism and pessimism. The upper channel line shows where bulls run out of steam, while the lower channel line shows where bears become exhausted. 

If a rally shoots out of a channel and prices close above it, it shows that the uptrend is exceptionally strong. When a rall fails to reach the upper channel line, it is a bearish sign, as it shows that bulls are becoming weaker. The reverse applies to downtrends, whereby the downtrend is strong and failure of channel breakout is a bullish sign. 


__Standard Deviation Channels (Bollinger Bands)__

The unique feature of these channels is that their width changes in response to market volatility. Their trading rules differ from those of regular channels:

1) Calculate a 21-day EMA (in this case period)
2) Subtract the 21-day Ema from each closing price to obtain all the deviations from the average
3) Square each of the deviations and sum to find the total squared deviations
4) Divide the total squared deviation by the EMA length to obtain the average squared deviation. 
5) Take the square root of the average squared deviation to obtain the standard deviation

A band becomes wider when the volatility increases but it narrows down when the volatility decreases. A narrow band identifies a quiet market. Bollinger bands help to identify transitions from quiet to active markets. 

In [ ]:
# recall 
# bars = ib.reqHistoricalData(contract = contract, endDateTime='', durationStr= '1 D', barSizeSetting='30 secs', whatToShow='TRADES', useRTH=False, keepUpToDate=True)

In [ ]:
# Constructing the Bollinger Bands

period = 21
df = util.df(bars)
df['EMA_21'] = df.close.ewm(span = period, adjust = False).mean()
df['SD'] = df.close.rolling(window=period).std()
df['Upper Band'] = df['EMA_21'] + (df['SD'] * 2)
df['Lower Band'] = df['EMA_21'] - (df['SD'] * 2)
df.dropna()

# def onBarUpdate(bars, hasNewBars):
#     clear_output()
#     display(df.tail(2).assign(New_bar = hasNewBars, Update_Time = datetime.now()))

# bars.updateEvent.clear()
# bars.updateEvent += onBarUpdate
# ib.sleep(60)
# bars.updateEvent.clear()


In [ ]:
def plotgraph(data):
        fig = go.FigureWidget()
        fig.add_trace(go.Scatter(
            x = data.date,
            y = data['close'],
            mode = 'lines',
            name = 'Close Price'
        ))
        fig.add_trace(go.Scatter(
            x = data.date,
            y = data['EMA_21'],
            mode = 'lines',
            name = '21 Period EMA',
            line = dict(color='blue')
        ))
        fig.add_trace(go.Scatter(
            x = data.date,
            y = data['Upper Band'],
            mode = 'lines',
            name = 'Upper Band',
            line = dict(color='red')
        ))
        fig.add_trace(go.Scatter(
            x = data.date,
            y = data['Lower Band'],
            mode = 'lines',
            name = 'Lower Band',
            line = dict(color='red')
        ))
        fig.add_trace(go.Scatter(
            x = data.date.tolist() + data.date.tolist()[::-1],
            y = data['Upper Band'].tolist() + data['Lower Band'].tolist()[::-1],
            fill ='toself',
            fillcolor = 'rgba(255,0,0,0.1)',
            line = dict(color='rgba(255,255,255,0)'),
            showlegend=False
        ))
        fig.update_layout(
            title = 'Bollinger Bands',
            xaxis_title = 'Index',
            yaxis_title = 'Price',
            legend_title = 'Legend',
            template = 'plotly_dark'
        )
        return fig

# CAUTION: Running this code will actually purchase make sure connection is to a paper trading account

In [ ]:
qty = 1
position = 0
period = 21
def onBarUpdate(bars, hasNewBar):
    global position
    df = util.df(bars)
    df['EMA_21'] = df.close.ewm(span = period, adjust = False).mean()
    df['SD'] = df.close.rolling(window=period).std()
    df['Upper Band'] = df['EMA_21'] + (df['SD'] * 1)
    df['Lower Band'] = df['EMA_21'] - (df['SD'] * 1)
    
    # BUY SIGNAL: Buy when close <= Lower Band + No position
    BUY_SIGNAL = (df['close'] <= df['Lower Band']).iloc[-1]*(position==0)
    # SHORT SIGNAL: Short when close >= Upper Band + No Position
    SHORT_SIGNAL = (df['close'] >= df['Upper Band']).iloc[-1]*(position==0)
    # CLOSE LONG: We have position>0 and close >= SMA
    CLOSE_LONG = (df['close'] >= df['EMA_21']).iloc[-1]*(position>0)
    # COVER SHORT: We have position < 0 and close <= SMA
    CLOSE_SHORT = (df['close'] <= df['EMA_21']).iloc[-1]*(position<0)

    if BUY_SIGNAL:
        print("BUY")
        trade = ib.placeOrder(contract=contract, order = MarketOrder(action = "BUY", totalQuantity=qty))
        position = qty
    elif SHORT_SIGNAL:
        print("SHORT")
        trade = ib.placeOrder(contract = contract, order = MarketOrder(action = "SELL", totalQuantity=qty))
        position = -qty
    elif CLOSE_LONG:
        print("Close Long")
        trade = ib.placeOrder(contract=contract, order = MarketOrder(action="SELL", totalQuantity=position))
    elif CLOSE_SHORT:
        print("Close Short")
        trade = ib.placeOrder(contract=contract, order = MarketOrder(action="BUY", totalQuantity=abs(position)))
    elif abs(position) > 0:
        print("Has position, hold")
    else:
        print("No position do nothing")
            
    clear_output(wait=True)
    display(df.tail(2))
    plotgraph(df)

        
bars.updateEvent+=onBarUpdate
ib.sleep(240)
bars.updateEvent.clear()


In [ ]:
bars.updateEvent.clear()

In [ ]:
ib.disconnect()

Now that the basic code and logic has been coded, how can this be done in an object orientated manner.

In [ ]:
class BollingerBands:
    ''' A live trading algorithm for a Bollinger Band Strategy using Interactive Brokers API.'''
    def __init__(self, symbol, qty, channel_width=2):
        self.symbol = symbol
        self.qty = qty
        self.channel_width = channel_width
        self.ib = None
        self.bars = None
        self.contract = None
        self.position = 0

        #Track Returns
        self.trades = []
        self.cumulative_return = 1.0
        self.cumulative_returns_history = [] 
        self.timestamps = []

        # Automatically start the trading process
        self.establish_connection()
        self.get_data()
        self.bars.updateEvent.clear()
        self.technical_indicators()
        self.fig = self.plotgraph(self.df)
        display(self.fig)
        self.start_trading()
        
    def establish_connection(self):
        '''Establish a connection to the Interactive Brokers API'''
        util.startLoop()
        self.ib = IB()
        try:
            self.ib.connect(port =  7497, clientId = 0)
            print(f"Connected to IB API")
        except Exception as e:
            print(f"Error connecting to IB: {e}")

    def get_data(self):
        '''Retrieve historical market data for the specified symbol.'''
        stock = Stock(symbol = self.symbol, exchange = 'SMART', currency = 'USD')
        qualified_contract = self.ib.qualifyContracts(stock) # self.contract
        self.contract = qualified_contract[0]
        self.bars = self.ib.reqHistoricalData(contract = self.contract, endDateTime='', durationStr='1 D', barSizeSetting='15 secs', whatToShow='TRADES', useRTH=False, keepUpToDate=True)
        self.df = util.df(self.bars)
        print("Data retrieved successfully.")

    def technical_indicators(self):
        '''Generate Bollinger Bands and other technical indicators for construction.'''
        self.df['EMA_21'] = self.df.close.ewm(span = 21, adjust = False).mean()
        self.df['SD'] = self.df.close.rolling(window = 21).std()
        self.df['Upper Band'] = self.df['EMA_21'] + (self.df['SD'] * self.channel_width)
        self.df['Lower Band'] = self.df['EMA_21'] - (self.df['SD'] * self.channel_width)
        self.df = self.df.dropna()
        print(f"Technical indicators calculated.")

    def generate_signals(self):
        '''Generate buy/sell signals based on Bollinger Bands.'''
        # BUY SIGNAL: Buy when close <= Lower Band + No position
        BUY_SIGNAL = (self.df['close'] <= self.df['Lower Band']).iloc[-1]*(self.position==0)
        # SHORT SIGNAL: Short when close >= Upper Band + No Position
        SHORT_SIGNAL = (self.df['close'] >= self.df['Upper Band']).iloc[-1]*(self.position==0)
        # CLOSE LONG: We have position>0 and close >= SMA
        CLOSE_LONG = (self.df['close'] >= self.df['EMA_21']).iloc[-1]*(self.position>0)
        # COVER SHORT: We have position < 0 and close <= SMA
        CLOSE_SHORT = (self.df['close'] <= self.df['EMA_21']).iloc[-1]*(self.position<0)

        return BUY_SIGNAL, SHORT_SIGNAL, CLOSE_LONG, CLOSE_SHORT
    
    def execute_trades(self):
        ''' Execute trades based on generated signals.'''
        BUY_SIGNAL, SHORT_SIGNAL, CLOSE_LONG, CLOSE_SHORT = self.generate_signals()
        current_time = self.df['date'].iloc[-1]
        wait_time = current_time + timedelta(seconds = 1)

        if BUY_SIGNAL:
            trade = self.ib.placeOrder(contract=self.contract, order = MarketOrder(action = "BUY", totalQuantity= self.qty))
            entry_price = self.ib.fills()[-1].execution.price 
            self.trades.append({'type': 'buy', 'price': entry_price, 'qty': self.qty, 'timestamp': current_time})
            print('=' * 55)
            print(f"Buying {self.qty} unit of {self.contract.symbol} at {entry_price}")
            print('=' * 55)
            self.position = self.qty

        elif SHORT_SIGNAL:
            trade = self.ib.placeOrder(contract = self.contract, order = MarketOrder(action = "SELL", totalQuantity=self.qty))
            entry_price = self.ib.fills()[-1].execution.price
            self.trades.append({'type': 'short', 'price': entry_price, 'qty': -self.qty, 'timestamp': current_time})
            print('=' * 55)
            print(f"Shorting {self.qty} unit of {self.contract.symbol} at {entry_price}")
            print('=' * 55)
            self.position = -self.qty

        elif CLOSE_LONG:
            trade = self.ib.placeOrder(contract= self.contract, order = MarketOrder(action="SELL", totalQuantity=self.position))
            exit_price = self.ib.fills()[-1].execution.price 
            entry_price = self.trades[-1]['price']
            percentage_return = ((exit_price - entry_price) / entry_price) * 100
            self.cumulative_return *= (1 + percentage_return / 100)
            profit = (exit_price - self.trades[-1]['price']) * self.position
            self.trades.append({'type': 'sell', 'price': exit_price, 'profit': profit, 'percentage_return': percentage_return, 'timestamp': current_time})
            self.cumulative_returns_history.append(self.cumulative_return)
            self.timestamps.append(current_time)
            self.position = 0
            print('=' * 55)
            print(f"Closing long position for {self.qty} unit of {self.contract.symbol} at an averge fill price of: {exit_price}, Profit : {profit}")
            print('=' * 55)
            self.position = 0

        elif CLOSE_SHORT:
            trade = self.ib.placeOrder(contract=self.contract, order = MarketOrder(action="BUY", totalQuantity=abs(self.position)))
            exit_price = self.ib.fills()[-1].execution.price
            entry_price = self.trades[-1]['price']
            percentage_return = ((entry_price - exit_price) / entry_price) * 100
            self.cumulative_return *= (1 + percentage_return / 100)
            profit = (self.trades[-1]['price'] - exit_price) * abs(self.position)
            self.trades.append({'type': 'cover', 'price': exit_price, 'profit': profit, 'percentage_return': percentage_return, 'timestamp': current_time})
            self.cumulative_returns_history.append(self.cumulative_return) 
            self.timestamps.append(current_time) 
            self.position = 0
            print('=' * 55)
            print(f"Covering short position for {self.qty} unit of {self.contract.symbol} at an average fill price of {trade.orderStatus.avgFillPrice}, Profit: {profit}")
            print('=' * 55)
            self.position = 0
        
        elif abs(self.position) > 0:
            if self.position > 0:
                print('=' * 55)
                print("Open Long position, Hold")
                print('=' * 55)
            elif self.position < 0:
                print('=' * 55)
                print("Open Short position, Hold")
                print('=' * 55)
        
        else:
            print('=' * 55)
            print("No position held, do nothing")
            print('=' * 55)

    def plotgraph(self, data):
        '''Function to plot a graph of the strategy'''
        fig = go.FigureWidget()
        fig.add_trace(go.Scatter(
            x = data.date,
            y = data['close'],
            mode = 'lines',
            name = 'Close Price'
        ))
        fig.add_trace(go.Scatter(
            x = data.date,
            y = data['EMA_21'],
            mode = 'lines',
            name = '21 Period EMA',
            line = dict(color='blue')
        ))
        fig.add_trace(go.Scatter(
            x = data.date,
            y = data['Upper Band'],
            mode = 'lines',
            name = 'Upper Band',
            line = dict(color='red')
        ))
        fig.add_trace(go.Scatter(
            x = data.date,
            y = data['Lower Band'],
            mode = 'lines',
            name = 'Lower Band',
            line = dict(color='red')
        ))
        fig.add_trace(go.Scatter(
            x = data.date.tolist() + data.date.tolist()[::-1],
            y = data['Upper Band'].tolist() + data['Lower Band'].tolist()[::-1],
            fill ='toself',
            fillcolor = 'rgba(255,0,0,0.1)',
            line = dict(color='rgba(255,255,255,0)'),
            showlegend=False
        ))

        buy_times = [trade['timestamp'] for trade in self.trades if trade['type'] == 'buy']
        buy_prices = [trade['price'] for trade in self.trades if trade['type'] == 'buy']
        fig.add_trace(go.Scatter(
            x=buy_times,
            y=buy_prices,
            mode='markers',
            name='Buy',
            marker=dict(color='green', size=10, symbol='triangle-up')))

        sell_times = [trade['timestamp'] for trade in self.trades if trade['type'] == 'sell']
        sell_prices = [trade['price'] for trade in self.trades if trade['type'] == 'sell']
        fig.add_trace(go.Scatter(
            x=sell_times,
            y=sell_prices,
            mode='markers',
            name='Sell',
            marker=dict(color='red', size=10, symbol='triangle-down')))

        short_times = [trade['timestamp'] for trade in self.trades if trade['type'] == 'short']
        short_prices = [trade['price'] for trade in self.trades if trade['type'] == 'short']
        fig.add_trace(go.Scatter(
            x=short_times,
            y=short_prices,
            mode='markers',
            name='Short',
            marker=dict(color='orange', size=10, symbol='triangle-down')))
        
        cover_times = [trade['timestamp'] for trade in self.trades if trade['type'] == 'cover']
        cover_prices = [trade['price'] for trade in self.trades if trade['type'] == 'cover']
        fig.add_trace(go.Scatter(
            x=cover_times,
            y=cover_prices,
            mode='markers',
            name='Cover',
            marker=dict(color='purple', size=10, symbol='triangle-up')))


        fig.update_layout(
            title = 'Bollinger Bands',
            xaxis_title = 'Index',
            yaxis_title = 'Price',
            legend_title = 'Legend',
            template = 'plotly_dark'
        )
        return fig
    
    def calculate_pcreturns(self):
        '''Calculate Percentage Returns for Strategy'''
        final_return_percentage = (self.cumulative_return - 1)*100
        print('=' * 55)
        print(f"Final Cumulative Return: {final_return_percentage:.2f}%")
        print('=' * 55)
        return final_return_percentage
    
    def plot_cumulative_returns(self):
        '''Plot Cumulative returns over time'''
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=self.timestamps,
            y=self.cumulative_returns_history,
            mode='lines+markers',
            name='Cumulative Returns',
            line=dict(color='blue'),
            marker=dict(size=6)))
        
        fig.update_layout(
            title='Cumulative Return Over Time',
            xaxis_title='Time',
            yaxis_title='Cumulative Return',
            template='plotly_dark')

        fig.show()

    def onBarUpdate(self, bars, hasNewBar):
        '''Callback for each bar update to recalculate indicators and execute trades.'''
        self.df = util.df(bars)
        self.technical_indicators()
        clear_output(wait=True)
        display(self.fig)
        self.update_plot()
        display(self.df.tail(2))
        self.execute_trades()

    def update_plot(self):
        '''Update the plot with the latest data.'''
        with self.fig.batch_update():
            self.fig.data[0].y = self.df['close']
            self.fig.data[1].y = self.df['EMA_21']
            self.fig.data[2].y = self.df['Upper Band']
            self.fig.data[3].y = self.df['Lower Band']
            self.fig.data[0].x = self.df['date']
            self.fig.data[1].x = self.df['date']
            self.fig.data[2].x = self.df['date']
            self.fig.data[3].x = self.df['date']
            self.fig.data[4].x = self.df['date'].tolist() + self.df['date'].tolist()[::-1]
            self.fig.data[4].y = self.df['Upper Band'].tolist() + self.df['Lower Band'].tolist()[::-1]

            buy_times = [trade['timestamp'] for trade in self.trades if trade['type'] == 'buy']
            buy_prices = [trade['price'] for trade in self.trades if trade['type'] == 'buy']
            self.fig.data[5].x = buy_times
            self.fig.data[5].y = buy_prices

            sell_times = [trade['timestamp'] for trade in self.trades if trade['type'] == 'sell']
            sell_prices = [trade['price'] for trade in self.trades if trade['type'] == 'sell']
            self.fig.data[6].x = sell_times
            self.fig.data[6].y = sell_prices

            short_times = [trade['timestamp'] for trade in self.trades if trade['type'] == 'short']
            short_prices = [trade['price'] for trade in self.trades if trade['type'] == 'short']
            self.fig.data[7].x = short_times
            self.fig.data[7].y = short_prices

            cover_times = [trade['timestamp'] for trade in self.trades if trade['type'] == 'cover']
            cover_prices = [trade['price'] for trade in self.trades if trade['type'] == 'cover']
            self.fig.data[8].x = cover_times
            self.fig.data[8].y = cover_prices

            print("Plot data updated") 

    
    def summary(self):
        '''Summary of trading results and cumulative returns visualised'''
        self.calculate_pcreturns()
        self.plot_cumulative_returns() 
            
    def start_trading(self):
        '''Start the trading process by subscribing the callback'''
        self.bars.updateEvent += self.onBarUpdate
        self.ib.sleep(180)
        self.summary()
        self.bars.updateEvent.clear()
        self.ib.disconnect()
        print("Disconnected from IB API. Trading stopped")

# Keyboard interrupt is Ctrl + C in console

In [3]:
trading_bot = BollingerBands(symbol='NVDA', qty=1, channel_width=0.5)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Close Price',
              'type': 'scatter',
              'uid': '5f4d701a-44b5-4d0a-972b-358dd8245df1',
              'x': array([datetime.datetime(2024, 11, 1, 4, 5, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')),
                          datetime.datetime(2024, 11, 1, 4, 5, 15, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')),
                          datetime.datetime(2024, 11, 1, 4, 5, 30, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')),
                          ...,
                          datetime.datetime(2024, 11, 1, 17, 22, 30, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')),
                          datetime.datetime(2024, 11, 1, 17, 22, 45, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern')),
                          datetime.datetime(2024, 11, 1, 17, 23, tzinfo=zoneinfo.ZoneInfo(key='US/Eastern'))],
                         dtype=object),
              'y': array([134.78, 134.8 , 134.8 , ..., 135.9 , 135.9 , 135.7 ])},

Plot data updated


,date,open,high,low,close,volume,average,barCount,EMA_21,SD,Upper Band,Lower Band
3212,2024-11-01 17:23:00-04:00,135.89,135.89,135.70,135.70,27714.0,135.770985,25,135.776111,0.141855,135.847038,135.705183
3213,2024-11-01 17:23:15-04:00,135.84,135.85,135.84,135.85,305.0,135.843279,3,135.782828,0.129157,135.847406,135.718249


Open Short position, Hold
Final Cumulative Return: 0.00%


Disconnected from IB API. Trading stopped


Status
1) Need to organise output while live - Done
2) Need to compute session returns - Done
3) Need to plot Equity - Done

This code is not meant to be a profitable trading algorithm it serves as an introduction for my own learning.
N.B the profits of the strategy while day trading on a low interval are diminished due to fixed costs of trading. This can be solved through either trading a stock which has a higher stock price, or by trading on a higher timeframe.

One other consideration is for simplicity of code as multiple methods are complex and difficult to read.

Stop loss needs to be added

## Status and Large Error Encountered

One major issue in the original code was capturing accurate entry and exit prices after placing a trade. This was tricky because of the method used to extract these prices. At first, I tried using entry_price = trade.orderStatus.avgFillPrice, but ran into a problem. There’s usually a slight delay (about half a second) between when an order is placed and when it’s actually filled. This delay caused the entry price to register as zero, because the order hadn’t completed yet, meaning entry prices weren’t properly stored, and percentage returns couldn’t be calculated.

To work around this, I first tried adding a half-second delay using ib.sleep() to give the trade time to fill. However, this approach conflicted with the asynchronous nature of the code, throwing off the process flow.

I also attempted to use the asyncio package to better handle the delay, but I didn’t have enough familiarity with asynchronous loops and couldn’t get it to work that way.

The best solution, which I plan to implement in live trading algorithms using IB_async, is to access the IB.fills list directly. This list provides the most recent fills, so by extracting the price from the latest filled contract, I can reliably capture the accurate entry and exit prices without disrupting the code’s asynchronous flow.

I will have to ammend this for the other live trading codes in my github: MACD and EMA stategies.